In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%load_ext cython

In [ ]:
import ROOT
import h5py
import os
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import datetime
import time

ROOT.gSystem.Load("$MARSSYS/libmars.so")

In [ ]:
filepath_in = "/remote/ceph/user/m/mhuetten/MAGIC_data/sorcerer/"
filepath_out = "/remote/ceph/user/m/mhuetten/MAGIC_data/ctapipe/"

filename_M1 = "20180821_M1_05074767.014_Y_CrabNebula-W0.40+035.root"
#filename_M2 = "20180821_M2_05074767.014_D_CrabNebula-W0.40+035.root"

ntels = 1

camgeom = ROOT.MGeomCamMagicTwo()
#print camgeom.GetPixels().At(4).Print() 

camera = ROOT.MHCamera(camgeom)

filenames = [filename_M1]#, filename_M2]
filepaths = [os.path.join(filepath_in, filename_M1)]#, os.path.join(filepath_in, filename_M2)]

infile_M1 = ROOT.TFile(os.path.join(filepath_in, filename_M1))
#infile_M2 = ROOT.TFile(os.path.join(filepath_in, filename_M2))

infiles = [infile_M1]#, infile_M2]

filename_out = ''.join(filename_M1.split("_M1"))

In [ ]:
ROOT.TBrowser()

In [ ]:
def get_ROOT_array(file, tree, branch, leaf):
    tfile = ROOT.TFile(file)
    ttree = getattr(tfile, tree)
    hist = ROOT.TH1D("hist","hist",100,-100,100)
    ttree.Project("hist", "%s.%s"%(branch,leaf),"","",100000000000,0)
    treeArr = ttree.GetV1()
    result = np.array(np.frombuffer(treeArr,count=int(hist.GetEntries())), dtype = np.double)
    return result

In [ ]:
f = h5py.File(os.path.join(filepath_out,filename_out.split(".root")[0] + ".hdf5"),"w")
dsets_events = []

for ntel in range(ntels):
    
    Runheader = ROOT.TChain("RunHeaders")
    Runheader.Add(os.path.join(filepath_in, filenames[ntel]))

    head = ROOT.MRawRunHeader()
    Runheader.SetBranchAddress("MRawRunHeader.",head)

    # extract global information out of first event:
    Runheader.GetEntry(0)
    TelescopeNumber =  head.GetTelescopeNumber()
    nevents = head.GetNumEvents()
    nevents = 2000
    #max_event = Events.GetEntries()
    nevents_check = 0
    
    RunNumber = head.GetRunNumber()
    RunType = head.GetRunTypeStr()
    if RunType == "Data":
        is_mc = False
    elif RunType == "Monte Carlo":
        is_mc = True

    
    Events = ROOT.TChain("Events")
    Events.Add(os.path.join(filepath_in, filenames[ntel]))
    
    evtdata_phe = ROOT.MCerPhotEvt()
    evtdata_arrival = ROOT.MArrivalTime()

    evtheader = ROOT.MRawEvtHeader()
 
    Events.SetBranchAddress("MCerPhotEvt.",evtdata_phe)
    Events.SetBranchAddress("MArrivalTime.",evtdata_arrival)
    Events.SetBranchAddress("MRawEvtHeader.",evtheader)
    
    
    arr_phe_2D = []
    arr_arrival_2D = []
    arr_triggerid = []

    DAQEvtNumber = []
    ClockCounter = []
    NumTrigLvl2 = []
    StereoEvtNumber = []
    TriggerID = []
    TrigPattern = []
    CalibrationPattern = []
    MacrocellsPattern = []
    TimeDiff = []    
    TrigPatternNames = []
    # valid for runs after 2013-12-13, see MARS MTriggerPatternDecode.cc, ll. 224        
    TrigPatternNames.append('L3 trigger')
    TrigPatternNames.append('Calibration')
    TrigPatternNames.append('LIDAR')
    TrigPatternNames.append('Pedestal')
    TrigPatternNames.append('Pulse injection')
    TrigPatternNames.append('Sumtrigger')
    TrigPatternNames.append('Topo trigger')
    TrigPatternNames.append('L1 trigger')
    
    
    start_time = time.time()

    
    for nevent in range(nevents):

        if nevent >= nevents:
           continue

        sys.stdout.write("\r ... processing event " + str(nevent) + ", " \
                         + "%s >>> Percent Done : %.2f%%" % (datetime.datetime.now(), 100*nevent/float(nevents)))

        Events.GetEvent(nevent)
        
        trigid = evtheader.GetTriggerID()
        
        binarypattern = '\t{0:016b}'.format(trigid&0xff) 
        
        triggpattern = (bool(int(binarypattern[-8])), 
                        bool(int(binarypattern[-1])), 
                        bool(int(binarypattern[-2])), 
                        bool(int(binarypattern[-3])), 
                        bool(int(binarypattern[-4])), 
                        bool(int(binarypattern[-5])), 
                        bool(int(binarypattern[-6])), 
                        bool(int(binarypattern[-7])))
                
        # check mono-character of run:
        #if nevent > 0:
         #   if triggpattern[0] == False:
         #       if StereoEvtNumber[-1] == StereoEvtNumber[-2]:
         #          StereoEvtNumber[-1] = -1
                #else:
                   # print "calib!"
        
        npixels = evtdata_phe.GetNumPixels()
        
        camera.SetCamContent(evtdata_phe)
        camera.SetAllUsed()
        arr_phe_evt = []
        for i_pixel in range(1184):
            arr_phe_evt.append(camera.GetBinContent(i_pixel + 1))

        camera.SetCamContent(evtdata_arrival)
        camera.SetAllUsed()
        arr_arrival_evt = []
        for i_pixel in range(1184):
            arr_arrival_evt.append(camera.GetBinContent(i_pixel + 1))
            
        if arr_phe_evt != []:
            arr_phe_2D.append(np.array(arr_phe_evt))
            arr_arrival_2D.append(np.array(arr_arrival_evt))
            
            DAQEvtNumber.append(evtheader.GetDAQEvtNumber())
            ClockCounter.append(evtheader.GetClockCounter())
            NumTrigLvl2.append(evtheader.GetNumTrigLvl2())
            StereoEvtNumber.append(evtheader.GetStereoEvtNumber())
            TriggerID.append(evtheader.GetTriggerID())
            TrigPattern.append(triggpattern)
            CalibrationPattern.append(evtheader.GetCalibrationPattern())
            MacrocellsPattern.append(evtheader.GetMacrocellsPattern())
            TimeDiff.append(evtheader.GetTimeDiff())
            
            
    end_time = time.time()
    elapsed = end_time - start_time
    print "\n File read into python arrays >>> Elapsed time: %.1f s" % elapsed
    
    ##########################
    # write data to hdf5 file:
    
    arr_phe_2D = np.array(arr_phe_2D)
    arr_arrival_2D = np.array(arr_arrival_2D)

    
    dsets_events.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/image", data=arr_phe_2D, compression='gzip'))#, compression_opts=9)
    dsets_events.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/peakpos", data=arr_arrival_2D, compression='gzip'))#, compression_opts=9)

    
f.close()

In [ ]:
f.close()